In [ ]:
#pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [2]:
from torchvision.models import alexnet, vgg16, resnet50, googlenet
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic=True

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from torchvision import datasets, models, transforms
Transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std = [0.229,0.224,0.225])])

CIFAR_train = datasets.CIFAR10(root='./data/train', train=True, download=True, transform=Transform)
CIFAR_test = datasets.CIFAR10(root='./data/test', train=False, download=True, transform=Transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 47676476.93it/s]


Extracting ./data/train/cifar-10-python.tar.gz to ./data/train


100%|██████████| 170498071/170498071 [00:03<00:00, 49103409.23it/s]


Extracting ./data/test/cifar-10-python.tar.gz to ./data/test


## AlexNet

In [ ]:
train_loader = DataLoader(CIFAR_train, batch_size=1000, shuffle=True)
test_loader = DataLoader(CIFAR_test, batch_size=1000)

In [ ]:
modelAlex = alexnet(pretrained=True) 

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 233MB/s]


In [ ]:
for param in modelAlex.parameters(): # по этим параметрам не будут браться градиенты, "заморозили" веса
    param.requires_grad = False

In [ ]:
modelAlex.classifier[6] = torch.nn.Linear(4096, 10)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modelAlex=modelAlex.to(device)

In [ ]:
modelAlex.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelAlex(test_data), dim=1)
    print('Точность (accuracy) на тестовой выборке без дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность (accuracy) на тестовой выборке без дообучения: 0.11


In [ ]:
optimizer = torch.optim.Adam(modelAlex.classifier[6].parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
modelAlex.train()
loss_history = []
EPOCHS = 10
for epoch in range(EPOCHS):
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.to(device)
        opt.zero_grad()
        loss_val = loss(modelAlex(batch), labels.to(device))
        loss_val.backward()
        opt.step()
        loss_history.append(loss_val.log().item())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
modelAlex.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelAlex(test_data), dim=1)
    print('Точность (accuracy) на тестовой выборке после дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность (accuracy) на тестовой выборке после дообучения: 0.845


## VGG

In [ ]:
train_loader = DataLoader(CIFAR_train, batch_size=100, shuffle=True)
test_loader = DataLoader(CIFAR_test, batch_size=100)

In [ ]:
modelVGG = vgg16(pretrained=True)
for param in modelVGG.parameters(): 
    param.requires_grad = False

modelVGG.classifier[6] = torch.nn.Linear(4096, 10)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modelVGG=modelVGG.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 232MB/s]


In [ ]:
modelVGG.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelVGG(test_data), dim=1)
    print('Точность на тестовой выборке без дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность на тестовой выборке без дообучения: 0.11


In [ ]:
optimizer = torch.optim.Adam(modelVGG.classifier[6].parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm.auto import tqdm
modelVGG.train()
loss_history = []
for epoch in tqdm(range(10)):
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.to(device)
        opt.zero_grad()
        loss_val = loss(modelVGG(batch), labels.to(device))
        loss_val.backward()
        opt.step()
        loss_history.append(loss_val.log().item())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
modelVGG.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelVGG(test_data), dim=1)
    print('Точность (accuracy) на тестовой выборке после дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность (accuracy) на тестовой выборке после дообучения: 0.88


## ResNet

In [ ]:
train_loader = DataLoader(CIFAR_train, batch_size=100, shuffle=True)
test_loader = DataLoader(CIFAR_test, batch_size=100)

In [ ]:
modelRES = resnet50(pretrained=True)
for param in modelRES.parameters(): 
    param.requires_grad = False
modelRES.fc = torch.nn.Sequential(torch.nn.Linear(in_features=2048, out_features=1000), 
                                  torch.nn.ReLU(inplace=True), 
                                  torch.nn.Linear(in_features=1000, out_features=10))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modelRES=modelRES.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
modelRES.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelRES(test_data), dim=1)
    print('Точность на тестовой выборке без дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность на тестовой выборке без дообучения: 0.08


In [ ]:
optimizer = torch.optim.Adam(modelRES.fc[2].parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
modelRES.train()
loss_history = []
for epoch in tqdm(range(10)):
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.to(device)
        opt.zero_grad()
        loss_val = loss(modelRES(batch), labels.to(device))
        loss_val.backward()
        opt.step()
        loss_history.append(loss_val.log().item())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
modelRES.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelRES(test_data), dim=1)
    print('Точность (accuracy) на тестовой выборке после дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность (accuracy) на тестовой выборке после дообучения: 0.82


## GoogleNet

In [ ]:
train_loader = DataLoader(CIFAR_train, batch_size=500, shuffle=True)
test_loader = DataLoader(CIFAR_test, batch_size=500)

In [ ]:
modelGoogle = googlenet(pretrained=True)
for param in modelGoogle.parameters(): 
    param.requires_grad = False
modelGoogle.fc = torch.nn.Sequential(torch.nn.Linear(in_features=1024, out_features=500), 
                                  torch.nn.ReLU(inplace=True), 
                                  torch.nn.Linear(in_features=500, out_features=10))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
modelGoogle=modelGoogle.to(device)

In [ ]:
modelGoogle.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelGoogle(test_data), dim=1)
    print('Точность на тестовой выборке без дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность на тестовой выборке без дообучения: 0.128


In [ ]:
optimizer = torch.optim.Adam(modelGoogle.fc[2].parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm.auto import tqdm
modelGoogle.train()
loss_history = []
for epoch in tqdm(range(10)):
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.to(device)
        opt.zero_grad()
        loss_val = loss(modelGoogle(batch), labels.to(device))
        loss_val.backward()
        opt.step()
        loss_history.append(loss_val.log().item())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
modelGoogle.eval()
with torch.no_grad():
    test_data, test_target = next(iter(test_loader))
    test_data = test_data.to(device)
    test_model_pred = torch.argmax(modelGoogle(test_data), dim=1)
    print('Точность (accuracy) на тестовой выборке после дообучения:', accuracy_score(test_target.cpu(), test_model_pred.cpu()))

Точность (accuracy) на тестовой выборке после дообучения: 0.714
